<a href="https://colab.research.google.com/github/gandalf1819/SF-Opioid-Crisis/blob/master/SF_drug_Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os

In [16]:
d_crime = pd.read_csv("/content/gdrive/My Drive/SF dataset/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")
d_crime.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId'],
      dtype='object')

In [0]:
np.random.seed(100)
random_d_crime=d_crime.sample(2215024)
train_size=int(0.67 *2215024)
train_df=random_d_crime[:train_size]
test_df=random_d_crime[train_size:]

In [18]:
train_df['DateTime'] = train_df.Date + " " + train_df.Time
train_df['DateTime'] = pd.to_datetime(train_df['DateTime'], format='%m/%d/%Y %H:%M')

test_df['DateTime'] = test_df.Date + " " + test_df.Time
test_df['DateTime'] = pd.to_datetime(test_df['DateTime'], format='%m/%d/%Y %H:%M')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [32]:
train_df.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId',
       'DateTime'],
      dtype='object')

In [33]:
test_df.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId',
       'DateTime'],
      dtype='object')

In [24]:
train_df.shape

(1484066, 14)

In [36]:
target = train_df["Category"].unique()
print(target.shape)
target

(39,)


array(['SUSPICIOUS OCC', 'LARCENY/THEFT', 'ASSAULT', 'NON-CRIMINAL',
       'VEHICLE THEFT', 'WARRANTS', 'BURGLARY', 'OTHER OFFENSES',
       'DRUG/NARCOTIC', 'MISSING PERSON', 'FRAUD', 'ROBBERY', 'VANDALISM',
       'KIDNAPPING', 'SEX OFFENSES, FORCIBLE', 'WEAPON LAWS',
       'DISORDERLY CONDUCT', 'RECOVERED VEHICLE', 'DRUNKENNESS',
       'SECONDARY CODES', 'PROSTITUTION', 'EMBEZZLEMENT', 'TRESPASS',
       'RUNAWAY', 'STOLEN PROPERTY', 'FORGERY/COUNTERFEITING',
       'LOITERING', 'ARSON', 'DRIVING UNDER THE INFLUENCE', 'LIQUOR LAWS',
       'EXTORTION', 'SUICIDE', 'FAMILY OFFENSES', 'BAD CHECKS', 'BRIBERY',
       'GAMBLING', 'SEX OFFENSES, NON FORCIBLE',
       'PORNOGRAPHY/OBSCENE MAT', 'TREA'], dtype=object)

In [38]:
# X = train_df.drop(train_df.columns[[1, 2, 5, 6]], axis = 1)
X = train_df
X.columns

Index(['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time',
       'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId',
       'DateTime'],
      dtype='object')

In [39]:
y = train_df.iloc[:, 1]
y.head()

1618683    SUSPICIOUS OCC
1303833     LARCENY/THEFT
38943       LARCENY/THEFT
1737093     LARCENY/THEFT
1490803           ASSAULT
Name: Category, dtype: object

In [0]:
def preprocess_data(dataset):
    dataset['DateTime'] = pd.to_datetime(dataset['DateTime'])
    dataset['Month'] = dataset.DateTime.apply(lambda x: x.month)
    dataset['Day'] = dataset.DateTime.apply(lambda x: x.day)
    dataset['Hour'] = dataset.DateTime.apply(lambda x: x.hour)
    dataset['Minute'] = dataset.DateTime.apply(lambda x: x.minute)
    dataset = dataset.drop('DateTime', 1)
    
    dataset = pd.get_dummies(data=dataset, columns=['DayOfWeek', 'PdDistrict'])
    return dataset

In [41]:
X = preprocess_data(X)
X.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,IncidntNum,Category,Descript,Date,Time,Resolution,Address,X,Y,Location,PdId,Month,Day,Hour,Minute,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,PdDistrict_BAYVIEW,PdDistrict_CENTRAL,PdDistrict_INGLESIDE,PdDistrict_MISSION,PdDistrict_NORTHERN,PdDistrict_PARK,PdDistrict_RICHMOND,PdDistrict_SOUTHERN,PdDistrict_TARAVAL,PdDistrict_TENDERLOIN
1618683,61097743,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,10/13/2006,23:13,NONE,200 Block of COLUMBUS AV,-122.406504,37.797678,POINT (-122.406503626072 37.7976778969458),6109774364070,10,13,23,13,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1303833,120517541,LARCENY/THEFT,GRAND THEFT PICKPOCKET,06/30/2012,17:40,NONE,16TH ST / FOLSOM ST,-122.415308,37.765316,POINT (-122.415308388816 37.765316135635),12051754106111,6,30,17,40,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
38943,146134967,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,07/06/2014,22:20,NONE,NORTHPOINT ST / POLK ST,-122.423658,37.805285,POINT (-122.423658369809 37.8052850926566),14613496706244,7,6,22,20,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1737093,81124398,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,10/18/2008,10:30,NONE,IRWIN ST / 8TH ST,-122.399679,37.766695,POINT (-122.399679247863 37.7666952934378),8112439806244,10,18,10,30,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1490803,100564255,ASSAULT,STALKING,04/10/2010,17:30,NONE,500 Block of LEAVENWORTH ST,-122.414645,37.786069,POINT (-122.414645409901 37.7860688364633),10056425515201,4,10,17,30,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [42]:
from sklearn.preprocessing import LabelEncoder
y = y.to_frame()
le = LabelEncoder()
y["Category"] = le.fit_transform(y["Category"])
print(y.head())

         Category
1618683        32
1303833        16
38943          16
1737093        16
1490803         1


In [43]:
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

{'ARSON': 0, 'ASSAULT': 1, 'BAD CHECKS': 2, 'BRIBERY': 3, 'BURGLARY': 4, 'DISORDERLY CONDUCT': 5, 'DRIVING UNDER THE INFLUENCE': 6, 'DRUG/NARCOTIC': 7, 'DRUNKENNESS': 8, 'EMBEZZLEMENT': 9, 'EXTORTION': 10, 'FAMILY OFFENSES': 11, 'FORGERY/COUNTERFEITING': 12, 'FRAUD': 13, 'GAMBLING': 14, 'KIDNAPPING': 15, 'LARCENY/THEFT': 16, 'LIQUOR LAWS': 17, 'LOITERING': 18, 'MISSING PERSON': 19, 'NON-CRIMINAL': 20, 'OTHER OFFENSES': 21, 'PORNOGRAPHY/OBSCENE MAT': 22, 'PROSTITUTION': 23, 'RECOVERED VEHICLE': 24, 'ROBBERY': 25, 'RUNAWAY': 26, 'SECONDARY CODES': 27, 'SEX OFFENSES, FORCIBLE': 28, 'SEX OFFENSES, NON FORCIBLE': 29, 'STOLEN PROPERTY': 30, 'SUICIDE': 31, 'SUSPICIOUS OCC': 32, 'TREA': 33, 'TRESPASS': 34, 'VANDALISM': 35, 'VEHICLE THEFT': 36, 'WARRANTS': 37, 'WEAPON LAWS': 38}


In [77]:
X['Category']=X['Category'].replace('NON CRIMINAL', 'NONCRIMINAL')
print(type(X['Category']))
y['Category']=y['Category'].replace('NON CRIMINAL', 'NONCRIMINAL')
print(type(y['Category']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [79]:
print(type(X_train), type(y_train))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [82]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth = 10, n_estimators = 256)
rf.fit(X_train.values, y_train.values.ravel())

ValueError: ignored

In [81]:
y_pred = rf.predict(X_test)
y_pred.shape
y_test.shape

from sklearn.metrics import accuracy_score
print ("Train Accuracy: ", accuracy_score(y_train, rf.predict(X_train)))
print ("Test Accuracy: ", accuracy_score(y_test, y_pred))

NotFittedError: ignored